# Trabalho Final Nb. 2 - Fundamentos de Big Data

CIMATEC 2025.05

Gustavo Campos

João Carneiro

Uziel Araújo

## Inicialização

In [1]:
# Importa bibliotecas
import pyspark.sql.functions as F
from pyspark.sql import SparkSession

In [2]:
# Inicia sessao spark
sc = SparkSession.builder.appName("SparkSQL") \
    .config("spark.jars.packages", "mysql:mysql-connector-java:8.0.29") \
    .getOrCreate()

## Carregamento

In [3]:
# Monta google drive para acesso ao dataset 'notafiscal.csv'
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Carrega como dataframe spark 'notafiscal.csv', salvo previamente na raiz do drive
path = "/content/drive/MyDrive/"
df = sc.read.csv(path + 'notafiscal.csv', inferSchema=True, header=True)

In [5]:
# Carrega dados de departamento gerados no nb. 1
df_dept = sc.read.csv(path + 'notafiscal_dept.csv', inferSchema=True, header=True)

df_dept.show(5)

+----------------+----------------+
|cod_departamento|nom_departamento|
+----------------+----------------+
|               6| Eletrodomestico|
|               3|       Papelaria|
|               7|          Movéis|
|               1|     Informática|
|               4|       cosmético|
+----------------+----------------+
only showing top 5 rows



## Seleção

In [6]:
# Carrega dataframe com os dados de produto
df_produto = df.select('cod_produto', 'nom_produto', 'cod_departamento').distinct()

df_produto.show(5)

+-----------+--------------------+----------------+
|cod_produto|         nom_produto|cod_departamento|
+-----------+--------------------+----------------+
|         13|              Estojo|               3|
|          7|         Celular 4Gb|               2|
|         17|Aparalho de Barbe...|               6|
|          9|        Celular  8GB|               2|
|          6|           Notebook |               1|
+-----------+--------------------+----------------+
only showing top 5 rows



## Transformação

In [7]:
# Combina as tabelas de produto e departamento
df_produto_dept = df_produto.join(df_dept, 'cod_departamento')

df_produto_dept.show(5)

+----------------+-----------+--------------------+----------------+
|cod_departamento|cod_produto|         nom_produto|nom_departamento|
+----------------+-----------+--------------------+----------------+
|               3|         13|              Estojo|       Papelaria|
|               2|          7|         Celular 4Gb|       Telefonia|
|               6|         17|Aparalho de Barbe...| Eletrodomestico|
|               2|          9|        Celular  8GB|       Telefonia|
|               1|          6|           Notebook |     Informática|
+----------------+-----------+--------------------+----------------+
only showing top 5 rows



In [8]:
# Transforma registros alterando seus nomes de produto
df_produto_dept = df_produto_dept.withColumn(
    'nom_produto',
    F.when(
        df_produto_dept.nom_produto == "Aparalho de Barbear Elétrico",
        "Barbeador Elétrico"
    ).otherwise(df_produto_dept.nom_produto)
)

df_produto_dept.show(5)

+----------------+-----------+------------------+----------------+
|cod_departamento|cod_produto|       nom_produto|nom_departamento|
+----------------+-----------+------------------+----------------+
|               3|         13|            Estojo|       Papelaria|
|               2|          7|       Celular 4Gb|       Telefonia|
|               6|         17|Barbeador Elétrico| Eletrodomestico|
|               2|          9|      Celular  8GB|       Telefonia|
|               1|          6|         Notebook |     Informática|
+----------------+-----------+------------------+----------------+
only showing top 5 rows



## Registro

Salva novos dataframes como csv no 'path' (diretório raiz do google drive)

In [9]:
# Salva csv do dataframe de departamento
df_produto_dept.toPandas().to_csv(path + 'notafiscal_produto.csv', index=False)